In [ ]:
!pip3 install torchsummary

In [1]:
import torch
import torchvision
import torch.nn as nn
import random
import os
from torchsummary import summary
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image

In [2]:
resize_shape = (200, 200)

transform_resize = torchvision.transforms.Compose([
    torchvision.transforms.Resize(resize_shape),
    torchvision.transforms.ToTensor()
])

In [3]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,csv_file,root_dir,transform=None):  
        self.whales = pd.read_csv(csv_file)
        self.whales_dict = self.whales.to_dict()
        self.image_names = list(self.whales_dict['Image'].values())
        self.image_ids = list(self.whales_dict['Id'].values())
        self.root_dir = root_dir
        self.transform = transform
        
    def __getitem__(self,idx):
        img0_tuple = random.choice(list(zip(self.image_names, self.image_ids)))
#         img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(list(zip(self.image_names, self.image_ids)))
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            img1_tuple = random.choice(list(zip(self.image_names, self.image_ids)))
        
        img0 = Image.open(self.root_dir+img0_tuple[0]).convert('RGB')
        img1 = Image.open(self.root_dir+img1_tuple[0]).convert('RGB')

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        sample = {'image_0': img0, 'image_1': img1, 'label': torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))}
        
#         return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
        return sample
                                                                              
    def __len__(self):
        return len(self.whales)

In [4]:
whale_dataset_1 = SiameseNetworkDataset(csv_file="/scratch/tmp/WhaleData/Data/train.csv", 
                             root_dir="/scratch/tmp/WhaleData/Data/train/",
                             transform=transform_resize)

In [5]:
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed = 42

dataset_size = len(whale_dataset_1)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(whale_dataset_1, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(whale_dataset_1, batch_size=batch_size,
                                                sampler=valid_sampler)

In [6]:
#checking what kind of system you are using
try:
  import google.colab
  from google.colab import drive
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False
try:
    hostname = !hostname
    if 'lab' in hostname[0] and '.eng.utah.edu' in hostname[0]:
        IN_CADE = True
    else:
        IN_CADE = False
except:
    IN_CADE = False

assert(not IN_CADE or not IN_COLAB)

#defining the folders where datasets will be, depending on the system
machine_being_used = 'cade' if IN_CADE else ('colab' if IN_COLAB else 'other')
pre_folder = '/scratch/tmp/' if machine_being_used == 'cade' else './'

In [7]:
def define_gpu_to_use(minimum_memory_mb = 3800):
    gpu_to_use = None
    try: 
        os.environ['CUDA_VISIBLE_DEVICES']
        print('GPU already assigned before: ' + str(os.environ['CUDA_VISIBLE_DEVICES']))
        return
    except:
        pass
    torch.cuda.empty_cache()
    for i in range(16):
        free_memory = !nvidia-smi --query-gpu=memory.free -i $i --format=csv,nounits,noheader
        if free_memory[0] == 'No devices were found':
            break
        free_memory = int(free_memory[0])
        if free_memory>minimum_memory_mb-500:
            gpu_to_use = i
            break
    if gpu_to_use is None:
        print('Could not find any GPU available with the required free memory of ' +str(minimum_memory_mb) + 'MB. Please use a different system for this assignment.')
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_to_use)
        print('Chosen GPU: ' + str(gpu_to_use))
        x = torch.rand((256,1024,minimum_memory_mb-500)).cuda()
        x = torch.rand((1,1)).cuda()
        del x

In [8]:
define_gpu_to_use()

Chosen GPU: 0


In [9]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
#             nn.Dropout2d(p=.2),
            
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(32),
#             nn.Dropout2d(p=.2),
            
            nn.AdaptiveAvgPool2d(output_size=(1,1))
        )

        self.fc1 = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),

            nn.Linear(512, 2046),
            nn.ReLU(),

            nn.Linear(2046, 4251)
        )

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [ ]:
model = SiameseNetwork()
model = model.cuda()
print (summary(model, (3, 200, 200)))

In [10]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [11]:
model = SiameseNetwork()
model.cuda()

criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
   
for epoch in range(10): 
    model.train() 
    losses = []
    print('Epoch ' + str(epoch)) 
    for sample in train_loader:
        print ("hello")
        optimizer.zero_grad() 
        image_0 = sample['image_0'].cuda()
        image_1 = sample['image_1'].cuda()
        label = sample['label'].cuda()
        out_0, out_1 = model(image_0, image_1)
        loss = criterion(out_0, out_1, label)
        loss.backward()
        optimizer.step() 
        losses.append(loss.item())
    print('loss: ' + str(np.mean(losses)))

Epoch 0
hello


RuntimeError: CUDA out of memory. Tried to allocate 1.12 GiB (GPU 0; 3.95 GiB total capacity; 3.15 GiB already allocated; 243.44 MiB free; 79.84 MiB cached)